In [1]:
import pandas as pd
from math import ceil
import numpy as np
import os

In [14]:
# 读入数据
# data = pd.read_csv('detect_pattern.csv', dtype={'手表编号':int})

In [ ]:
if len(data['手表编号'].unique()) == 0:
    raise Exception("数据为空")
elif len(data['手表编号'].unique()) > 1:
    raise Exception("存在多个设备的数据")
else:
    print('开始判定模式')

In [2]:
def smoothingList(array):
    length = len(array)
    if length<9:
        return array
    somoothedArr = []
    y4 = (5950*array[0] + 1960*array[1] - 140*array[2] - 840*array[3] - \
          630*array[4] + 0*array[5] + 560*array[6] + 560*array[7] - 490*array[8]) / 6930
    y3 = (1960*array[0] + 2275*array[1] + 1960*array[2] + 1260*array[3] + \
          420*array[4] - 315*array[5] - 700*array[6] - 490*array[7] + 560*array[8]) / 6930
    y2 = (-140*array[0] + 1960*array[1] + 2575*array[2] + 2160*array[3] + \
          1170*array[4] + 60*array[5] - 715*array[6] - 700*array[7] + 560*array[8]) / 6930
    y1 = (-840*array[0] + 1260*array[1] + 2160*array[2] + 2175*array[3] + \
          1620*array[4] + 810*array[5] + 60*array[6] - 315*array[7] + 0*array[8]) / 6930
    
    y_4 = (5950*array[length-1] + 1960*array[length-2] - 140*array[length-3] - 840*array[length-4] - \
          630*array[length-5] + 0*array[length-6] + 560*array[length-7] + 560*array[length-8] - 490*array[length-9]) / 6930
    y_3 = (1960*array[length-1] + 2275*array[length-2] + 1960*array[length-3] + 1260*array[length-4] + \
          420*array[length-5] - 315*array[length-6] - 700*array[length-7] - 490*array[length-8] + 560*array[length-9]) / 6930
    y_2 = (-140*array[length-1] + 1960*array[length-2] + 2575*array[length-3] + 2160*array[length-4] + \
          1170*array[length-5] + 60*array[length-6] - 715*array[length-7] - 700*array[length-8] + 560*array[length-9]) / 6930
    y_1 = (-840*array[length-1] + 1260*array[length-2] + 2160*array[length-3] + 2175*array[length-4] + \
          1620*array[length-5] + 810*array[length-6] + 60*array[length-7] - 315*array[length-8] + 0*array[length-9]) / 6930
    somoothedArr.append(y4)
    somoothedArr.append(y3)
    somoothedArr.append(y2)
    somoothedArr.append(y1)
    for i in range(4, length-4):
        y = (-630*array[i-4] + 420*array[i-3] + 1170*array[i-2] + 1620*array[i-1] + \
             1770*array[i] +1620*array[i+1] + 1170*array[i+2] + 420*array[i+3] - 630*array[i+4]) / 6930
        somoothedArr.append(y)
    somoothedArr.append(y_1)
    somoothedArr.append(y_2)
    somoothedArr.append(y_3)
    somoothedArr.append(y_4)
    return somoothedArr

In [3]:
### 对-120重新赋值
def handle_invalid(df):
    df_copy = df.copy()
    invalid_index = df_copy[df_copy['校内天线信号强度']==-120].index.values
    for idx in invalid_index[invalid_index>0]:
        df_copy.loc[idx, '校内天线信号强度'] = df_copy.loc[idx-1, '校内天线信号强度']
    invalid_index = df_copy[df_copy['校外天线信号强度']==-120].index.values
    for idx in invalid_index[invalid_index>0]:
        df_copy.loc[idx, '校外天线信号强度'] = df_copy.loc[idx-1, '校外天线信号强度']    
    return df_copy

In [90]:
def prepare_df(df):
#     valid_df = handle_invalid(df)
    valid_df = df
    # 平滑原始数据
    valid_df['校内天线信号强度'] = smoothingList(valid_df['校内天线信号强度'])
    valid_df['校外天线信号强度'] = smoothingList(valid_df['校外天线信号强度'])
    valid_df['新信号强度差值'] = valid_df['校内天线信号强度'] - valid_df['校外天线信号强度']
    df_count = valid_df['手表编号'].value_counts().sort_index(ascending=True)
    return valid_df, df_count

## 提取特征

In [19]:
def add_feature(valid_df, df_count):
    ### 越过‘0’线的次数和越过‘0’线时左右3个点的乘积
    cnt_list = []
    left_list = []
    right_list = []
    for i in range(len(df_count.index)):
        cnt = 0
        left = []
        right = []
        viewed_df = valid_df[valid_df['手表编号'] == df_count.index[i]].reset_index()
        for j in range(1, viewed_df.shape[0]):
            product = viewed_df['新信号强度差值'][j-1] * viewed_df['新信号强度差值'][j]
            if product < 0:
                cnt += 1
                left.append(viewed_df['新信号强度差值'][j-1])
                right.append(viewed_df['新信号强度差值'][j])

        cnt_list.append(cnt)
        left_list.append(left)
        right_list.append(right)

    ### 前面和后面分别加和
    add_front_list = []
    add_back_list = []
    front_is_positive = []
    back_is_positive = []
    for i in range(len(df_count.index)):
        viewed_df = valid_df[valid_df['手表编号'] == df_count.index[i]].reset_index()
        add_front = ceil(sum(viewed_df['新信号强度差值'].tolist()[0:5]))
        add_back = ceil(sum(viewed_df['新信号强度差值'].tolist()[-5:]))
        front_is_positive.append(1 if add_front >= 0 else 0)
        back_is_positive.append(1 if add_back >= 0 else 0)
        add_front_list.append(add_front)
        add_back_list.append(add_back)

    feature_df = pd.DataFrame()
    feature_df['cnt_list'] = cnt_list
    feature_df['cnt_list'] = cnt_list
    feature_df['add_front_list'] = add_front_list
    feature_df['add_back_list'] = add_back_list
    feature_df['front_is_positive'] = front_is_positive
    feature_df['back_is_positive'] = back_is_positive
    return feature_df

In [20]:
feature_df = pd.DataFrame()
valid_df, df_count = prepare_df(data)
feature_df = feature_df.append(add_feature(valid_df, df_count))

In [9]:
if feature_df['front_is_positive'].tolist()[0] > 0.5:
    if feature_df['back_is_positive'].tolist()[0] > 0.5:
        flag = '校内'
    else:
        flag = '出校'
else:
    if feature_df['back_is_positive'].tolist()[0] > 0.5:
        flag = '进校'
    else:
        flag = '校外'

In [10]:
ans = pd.DataFrame({'手表编号':data['手表编号'][0], '接收时间':data['接收时间'].values[-1], '状态':flag}, 
                   index=np.arange(1))

In [11]:
if not os.path.exists('result.csv'):
    open('result.csv', 'a').close()
    result = pd.read_csv('result.csv', names=['手表编号', '接收时间', '状态'], dtype={'手表编号':int})
    result.to_csv('result.csv', index=False, encoding='utf-8')
result = pd.read_csv('result.csv', dtype={'手表编号':int})
result = pd.concat([result, ans], ignore_index=True).reset_index(drop=True)
result.to_csv('result.csv', index=False, encoding='utf-8')

## 验证

In [117]:
# 读入excel
data = pd.read_excel('origin_data/2017-05-12/校外(15:51:20_15:54:05).xls', 
                     names=['手表编号','接收时间','校内天线信号强度','校外天线信号强度','信号强度差值'])

In [118]:
feature_df = pd.DataFrame()
valid_df, df_count = prepare_df(data)
feature_df = feature_df.append(add_feature(valid_df, df_count))

In [119]:
feature_df[26:]

,cnt_list,add_front_list,add_back_list,front_is_positive,back_is_positive
26,10,-129,-107,0,0
27,10,-70,-135,0,0
28,12,-153,-26,0,0
29,6,-202,-171,0,0
30,10,-121,-130,0,0
31,8,-183,-90,0,0
32,6,-170,-118,0,0
33,10,-60,-109,0,0
34,8,-164,-170,0,0
35,14,-95,-181,0,0


In [75]:
# print(feature_df['front_is_positive'].tolist(), feature_df['back_is_positive'].tolist())

出校(15:16:12_15:17:54).xls
40
出校(15:29:30_15:31:58).xls
28  67
出校(15:38:50_15:42:16).xls
6 7 11 27 32 33 36 48 59 63 71
出校(15:46:30_15:49:26).xls
15 29 

校外(15:51:20_15:54:05).xls
13 15 20 45 67 78
校外(15:55:16_15:57:41).xls
21 51
校外(16:24:30_16:29:30).xls
2 3 17 26 69
进校(15:33:30_15:36:44).xls
38 66 
进校(15:42:26_15:46:03).xls
29 52 56

In [85]:
valid_df[valid_df['手表编号']==1]

,手表编号,接收时间,校内天线信号强度,校外天线信号强度,信号强度差值,新信号强度差值
0,1,2017-05-12 15:51:25,-116.393939,-82.424242,-38,-33.969697
1,1,2017-05-12 15:51:26,-96.878788,-82.984848,-6,-13.893939
2,1,2017-05-12 15:51:28,-86.580087,-82.989177,-36,-3.590909
3,1,2017-05-12 15:51:29,-83.028139,-82.649351,35,-0.378788
4,1,2017-05-12 15:51:30,-83.753247,-82.177489,-40,-1.575758
5,1,2017-05-12 15:51:32,-85.779221,-81.870130,33,-3.909091
6,1,2017-05-12 15:51:36,-85.419913,-82.281385,-37,-3.138528
7,1,2017-05-12 15:51:37,-84.584416,-82.484848,0,-2.099567
8,1,2017-05-12 15:51:38,-82.649351,-82.316017,0,-0.333333
9,1,2017-05-12 15:51:39,-80.324675,-81.450216,1,1.125541
